In [1]:
import pandas as pd

In [2]:
point_panel_in = "../data/final/point_panel.parquet"
tract_panel_in = "../data/final/tract_panel.parquet"

panel_out = "../data/final/model_data.parquet"

# Pipeline In

In [3]:
point_panel = pd.read_parquet(point_panel_in)
tract_panel = pd.read_parquet(tract_panel_in)

In [4]:
tract_panel = tract_panel.loc[~tract_panel.is_weekend]
point_panel = point_panel.loc[~point_panel.is_weekend]

In [5]:
def get_model_data(panel, transit):
    model_data = panel.query(f'transit == "{transit}"')
    empty_units = model_data.groupby('id')['rides'].transform('sum') == 0
    model_data = model_data[~empty_units]
    return model_data

In [6]:
model_data = pd.concat([
    get_model_data(tract_panel, "uber").assign(id = lambda x: x['id'].astype(str)),
    get_model_data(point_panel, "train").assign(id = lambda x: x['id'].astype(str)),
    get_model_data(point_panel, "bike").assign(id = lambda x: x['id'].astype(str)),
])

# Pipeline Out

In [7]:
model_data.to_parquet(panel_out, index=False)